### 【 데이터 전용 Dataset/DataLoader 】
- pytorch에서 데이터 관리 및 유지보수를 위한 클래스 제공
- Dataset    : 사용자 데이터에 맞게 커스텀 클래스 생성
- DataLoader : 배치크기만큼 데이터를 추출해 주는 역할
- 데이터셋 분리 => random_split() 함수 제공 : 타겟 클래스 고려하지 않은 랜덤한 데이터 분리
- 층화기법 => train_test_split() 함수 : scikit-learn의 함수

[1] 모듈 로딩 및 데이터 준비 <hr>

In [51]:
# [1-1] 모듈 로딩
import pandas as pd
import torch                                        # 텐서 및 수치과학 함수들 관련 모듈
from torch.utils.data import Dataset, DataLoader    # pytorch의 데이터 로딩
from torch.utils.data import Subset                 # pytorch의 데이터셋 관련 모듈
from sklearn.model_selection import train_test_split

In [52]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

# [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE, header=None)
testDF  = pd.read_csv(TEST_FILE, header=None)

[2] 커스텀 데이터셋 클래스 생성 및 데이터 적용 <hr>

In [53]:
# -------------------------------------------------------------------------------------
# [2-2] 커스텀 데이터셋 클래스 정의
# -------------------------------------------------------------------------------------
# 클래스이름 : ClfDataset
# 부모클래스 : Dataset
# 오버라이딩 : _ _init_ _(self)         : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
#            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
#            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
#                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
# -------------------------------------------------------------------------------------
class ClfDataset(Dataset):

    #- 피쳐와 타겟 저장 및 기타 속성 초기화
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    #- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0]
    
    #- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.float32)
        return xTS, yTS
    

In [54]:
# -------------------------------------------------------------------------------------
# [2-3] 커스텀 데이터셋 인스턴스 생성 및 사용
# -------------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)   ## <= trainDS, validDS 분리
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 60000,  testDS : 10000


In [55]:
# -------------------------------------------------------------------------------------
# [2-4] 학습용/검증용/테스트용 데이터셋 분리
# -------------------------------------------------------------------------------------
# 학습용   : 순수 학습에 즉, 데이터셋에 규칙/패턴을 찾기 위한 데이터셋
# 검증용   : 제대로 데이터셋에서 규칙/패턴을 찾는지 확인 용도
#           에포크 단위로 찾은 규칙/패턴의 검증용으로 사용
# 테스트용 : 데이터셋에 규칙/패턴 찾은 후 최종 테스트용으로 사용

# 학습용 데이터셋의 개수
print(f'allDS   :{len(allDS)}개, testDS : {len(testDS)}개')

# 학습용 데이터셋에서 타겟/라벨만 추출
targetList = allDS.y

# 학습용 데이터셋에서 타겟/라벨만 추출
dataNP = allDS.y.shape

# 학습용 데이터셋에서 타겟/라벨 인덱스 생성
dataIndexList = list(range(len(allDS)))

allDS   :60000개, testDS : 10000개


In [56]:
# 학습용/검증용 데이터셋 인덱스 분리
# - train_test_split() 함수 : train:test : 75:27 비율로 학습용, 테스트용 데이터셋 분리
#                            stratify : 분류용 데이터셋의 경우 카테고리 비율 유지해서 데이터셋 분리
X_trainIdx, X_validIdx, y_train, y_valid = train_test_split(dataIndexList,
                                                            targetList,
                                                            train_size=0.8,
                                                            stratify=targetList,
                                                            random_state=10)

print(f"X_trainIdx : {len(X_trainIdx)}개, {type(X_trainIdx)}")
print(f"X_validIdx : {len(X_validIdx)}개, {type(X_validIdx)}")
print(f"testDS     : {len(testDS)}개")

X_trainIdx : 48000개, <class 'list'>
X_validIdx : 12000개, <class 'list'>
testDS     : 10000개


In [57]:
# -------------------------------------------------------------------------------------
# 학습용/검증용 데이터셋 생성 ----> Dataset ---> 2개 Subset 분리
# -------------------------------------------------------------------------------------
trainDS = Subset(allDS, X_trainIdx)
validDS = Subset(allDS, X_validIdx)

print(f"allDS   : {len(allDS)}개, {type(allDS)}")
print(f"trainDS : {len(trainDS)}개, {type(trainDS)}")
print(f"validDS : {len(validDS)}개, {type(validDS)}")

allDS   : 60000개, <class '__main__.ClfDataset'>
trainDS : 48000개, <class 'torch.utils.data.dataset.Subset'>
validDS : 12000개, <class 'torch.utils.data.dataset.Subset'>
